# Test Azure GPT-4 based model with RAG store of Top 500 Bioconductor package

### Get relevant keys from Azure

In [1]:
import os, openai, requests
    
## Declare keys
api_key = os.getenv("OPENAI_API_KEY")
search_key = os.getenv("OPENAI_SEARCH_KEY")
endpoint = os.getenv("OPENAI_ENDPOINT")

Declare the deployment name, endpoint and the name of the search index.

In [2]:

openai.api_type = "azure"
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"

# Azure OpenAI setup
openai.api_base = endpoint
openai.api_key = api_key 
deployment_id = "gpt-4-test" 

# Azure AI Search setup
search_endpoint = "https://hmsaisearch.search.windows.net"; # Add your Azure AI Search endpoint here
search_index_name = "bioc-top-500"; # Add your Azure AI Search index name here
    
# https://hms-it-openai.openai.azure.com/openai/deployments/gpt-4-test/extensions/chat/completions?api-version=2024-02-15-preview

Create a "client" object with AzureOpenAI with the correct api key.

In [3]:
# https://hms-it-openai.openai.azure.com/openai/deployments/gpt-4-test/extensions/chat/completions?api-version=2024-02-15-preview

client = openai.AzureOpenAI(
    base_url=f"{endpoint}/openai/deployments/{deployment_id}/extensions",
    api_key=api_key,
    api_version="2023-08-01-preview",
)

Note: This step is to set up the RAG store i.e "bring your own data" search index that has to be registered with the deployment in hand - `gpt-4-test`.

In [4]:
def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
 
    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

     def send(self, request, **kwargs):
         request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
         return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

setup_byod(deployment_id)

### Generic function to ask a question from our BYOD model with GPT-4

In [5]:
def ask_question(question):

    completion = client.chat.completions.create(
        messages=[{"role": "user", "content": question}],
        model=deployment_id,
        extra_body={
            "dataSources": [
                {
                    "type": "AzureCognitiveSearch",
                    "parameters": {
                        "endpoint": search_endpoint,
                        "key": search_key,
                        "indexName": search_index_name,
                        "roleInformation": "Act as an expert in the R programming language and the Bioconductor suite of packages.  ​\n\nYour job is to advise users on the usage of the various Bioconductor packages considering the documents you have in the data store.  ​\n\nTo complete this task, you can use the data you have stored that contain the vignettes of all the packages in Bioconductor and all the reference files of every function in every package of Bioconductor. ​You may also answer some general R, general programming, or Biomedical information.\n\nIf you do not know the answer ask the user to refer to https://bioconductor.org. \n\nAdd a disclaimer at the end of each response saying this model works only on the top 500 most used Bioconductor packages."
                    }
                }
            ]
        }
    )

    print(f"{completion.choices[0].message.role}: {completion.choices[0].message.content}")

    return

In [6]:
# `context` is in the model_extra for Azure
# print(f"\nContext: {completion.choices[0].message.model_extra['context']['messages'][0]['content']}")

### Questions for SummarizedExperiemnt

In [11]:
## summarized experiment 
question = "How many classes are there in the Summarized Experiment package? Just give me a number."

ask_question(question)

assistant: The SummarizedExperiment package contains two classes: SummarizedExperiment and RangedSummarizedExperiment [doc1].


### Questions for DESeq2

In [8]:
## DESeq2

question = "DESeq2 performs normalization by estimating size factors for each sample. If your experiment has 5 samples, how many size factors will DESeq2 estimate?"

ask_question(question)

assistant: DESeq2 estimates one size factor for each sample in your experiment. Therefore, if your experiment has 5 samples, DESeq2 will estimate 5 size factors [doc1]. 

Please note that this model works only on the top 500 most used Bioconductor packages.


### Questions for limma

In [9]:
## Limma
question = "You use limma to analyze RNA-seq data from a case-control study with 30 control samples and 30 case samples. After fitting the linear model, how many coefficients will be estimated by limma for the gene expression data (assuming no additional covariates are included in the model)?"

ask_question(question)

assistant: In a case-control study design with no additional covariates, the linear model in limma will estimate two coefficients for each gene: one for the intercept and one for the group effect (case vs control). The intercept represents the baseline gene expression level (usually corresponding to the control group), and the group effect represents the difference in gene expression between the case and control groups [doc2][doc4].

Please note that this model works only on the top 500 most used Bioconductor packages. For more complex models or less common packages, please refer to the specific package documentation or https://bioconductor.org.


### Questions for SingleCellExperiment

This questions doesn't give a good answer.

In [10]:
question = "how many slots does the SingleCellExperiment object have?"

ask_question(question)

assistant: The SingleCellExperiment object is a complex object with multiple slots. However, the exact number of slots is not specified in the retrieved documents. The SingleCellExperiment object is designed to store single-cell experiment data, including assay data, feature data, and cell metadata. It is a part of the Bioconductor project and is used extensively in R packages for single-cell data analysis.

For detailed information about the structure and slots of the SingleCellExperiment object, you can refer to the official Bioconductor documentation at https://bioconductor.org.

Please note that this model works only on the top 500 most used Bioconductor packages.
